In [1]:
from cmath import nan
# from unicodedata import name
# from matplotlib import projections
# from matplotlib.pyplot import table
import pandas as pd
import os
import glob
from pymongo import MongoClient, ASCENDING, DESCENDING, errors
from datetime import date, timedelta, datetime, timezone
from flask import Flask, jsonify, request, redirect, Response, send_file
from flask_cors import CORS
import json
import math
# from urllib.parse import urlparse
# from urllib. parse import parse_qs
from flask_cors import CORS, cross_origin
from flask import send_from_directory
from pandas.tseries.offsets import MonthEnd
from werkzeug .utils import secure_filename
from zipfile import ZipFile
from svs_report import svsreport
from svs_report import gen_all_letters
from svs_report import gen_excel
import shutil
from bson import json_util, ObjectId